In [1]:
from music21 import stream, interval, corpus, instrument, pitch
from music21 import converter, note, chord, environment, duration, key
import notebook
import argparse, string, pathlib
import pandas as pd
from sklearn import preprocessing
import numpy as np
import json, re, random
from dwbzen.music import MusicUtils, Instruments


In [2]:
filename = "/Compile/careers/resources/editions.json"
fp = open(filename, "r")
jtxt = fp.read()
editions = json.loads(jtxt)
editions

{'editions': ['Hi-Tech', 'UK'],
 'Hi-Tech': {'version': '1.1',
  'full_name': 'Hi Tech Edition',
  'Help': 'Rule Book v2 booklet.pdf',
  'Game Parameters': 'gameParameters_Hi-Tech.json',
  'Opportunity Cards': 'opportunityCards_Hi-Tech.json',
  'Experience Cards': 'experienceCards_Hi-Tech.json',
  'Occupations': 'occupations_Hi-Tech.json',
  'Players': 'players_Hi-Tech.json',
  'Layout': 'gameLayout_Hi-Tech.json',
  'Rules': 'rules.json',
  'configuration': {'starting_salary': 2000, 'initial_cash': 2000}},
 'UK': {'version': '1.0',
  'full_name': 'UK Edition: Destination London',
  'Help': 'Rule Book Destination London.pdf',
  'Game Parameters': 'gameParameters_UK.json',
  'Opportunity Cards': 'opportunityCards_UK.json',
  'Experience Cards': 'experienceCards_UK.json',
  'Occupations': 'occupations_UK.json',
  'Players': 'players_UK.json',
  'Layout': 'gameLayout_UK.json',
  'Rules': 'rules_UK.json',
  'configuration': {'starting_salary': 2000, 'initial_cash': 2500}}}

In [6]:
print(editions.keys())
print(editions['Hi-Tech'])
print(editions['editions'])

dict_keys(['editions', 'Hi-Tech', 'UK'])
{'version': '1.1', 'full_name': 'Hi Tech Edition', 'Help': 'Rule Book v2 booklet.pdf', 'Game Parameters': 'gameParameters_Hi-Tech.json', 'Opportunity Cards': 'opportunityCards_Hi-Tech.json', 'Experience Cards': 'experienceCards_Hi-Tech.json', 'Occupations': 'occupations_Hi-Tech.json', 'Players': 'players_Hi-Tech.json', 'Layout': 'gameLayout_Hi-Tech.json', 'Rules': 'rules.json', 'configuration': {'starting_salary': 2000, 'initial_cash': 2000}}
['Hi-Tech', 'UK']


In [12]:
'Hi-Tech' in editions['editions']

True

In [27]:
fp = open("/Compile/careers/resources/occupations_Hi-Tech.json", "r")
occupations = json.loads(fp.read())
occupations['occupations']

['FMC',
 'UF',
 'Amazon',
 'ESPN',
 'Disney',
 'ListerAndBacon',
 'Facebook',
 'Google',
 'Pfizer',
 'SpaceX']

In [7]:
fp = open("/Compile/careers/resources/gameParameters_UK.json", "r")
jtxt = fp.read()
game_parameters = json.loads(jtxt)
game_parameters

{'starting_salary': 2000,
 'starting_cash': 2000,
 'currency': 'Pounds',
 'currency_symbol': 'Â£',
 'starting_experience_cards': 0,
 'starting_opportunity_cards': 0,
 'default_game_points': 100,
 'timed_game': 0,
 'default_game_minutes': 60}

In [28]:
# occupation in its own JSON file
# 
fp = open("/Compile/careers/resources/UF_Hi-Tech.json", "r")
jtxt = fp.read()
occupation = json.loads(jtxt)
occupation['name']

'UF'

In [22]:
print(occupation['configuration'])

{'occupationClass': 'college', 'fullName': 'University of Florida', 'background_image': 'occupation_entrance.png', 'image': 'UF logo.jpg', 'text': 'Must have: $500 tuition<br/>Degrees available:<br/>**Aerospace, Agronomy, Business Admin, Computer Science, Hospitality, Journalism, Law, Marketting and Pharmacy**', 'entryFee': 500, 'degrees': ['Aerospace', 'Agronomy', 'Business Admin', 'Computer Science', 'Hospitality', 'Journalism', 'Law', 'Marketing', 'Pharmacy'], 'maxDegrees': 4}


In [23]:
occupation['occupationSquares'][0:2]

[{'number': 0,
  'background_image': 'occupation_square.png',
  'text': 'You need serious counseling',
  'image': 'frown_emoji.png',
  'stars': 0,
  'hearts': 0,
  'bonus': 0,
  'experience': 0,
  'opportunities': 0,
  'action_text': 'Lose next turn',
  'specialProcessing': 'loseNextTurn'},
 {'number': 1,
  'background_image': 'occupation_square.png',
  'text': 'Attend Group Love concert',
  'image': '',
  'stars': 0,
  'hearts': 2,
  'bonus': 0,
  'experience': 0,
  'opportunities': 0,
  'action_text': '',
  'specialProcessing': {}}]

In [24]:
occupation['occupationSquares'][2]['specialProcessing']

{'type': 'shortcut', 'next_square': 9}

In [19]:
fp = open("/Compile/careers/resources/gameLayout_Hi-Tech.json", "r")
jtxt = fp.read()
hitech_layout = json.loads(jtxt)
hitech_layout

{'Comments': 'Board layout for the Hi-Tech version.',
 'layout': [{'number': 0,
   'name': 'Payday',
   'type': 'corner_square',
   'background_image': 'corner_square.png',
   'image': 'startHereAndArrow.jpg',
   'text': 'Collect Salary<br/>as you pass<p/><h1>PAYDAY</h1><h2>Collect DOUBLE if you land here</h2>',
   'specialProcessing': {'type': 'payday', 'amount': '$Salary'}},
  {'number': 1,
   'name': 'Opportunity',
   'type': 'opportunity_square',
   'background_image': 'opportunity.png',
   'text': '**OPPORTUNITY KNOCKS**',
   'specialProcessing': {'type': 'opportunity'}},
  {'number': 2,
   'name': 'IncomeTax',
   'type': 'danger_square',
   'background_image': 'danger_border_square.png',
   'text': '**INCOME TAX DUE**',
   'specialProcessing': {'type': 'incomeTax', 'amount': '$Salary'}},
  {'number': 3,
   'name': 'Amtrak',
   'type': 'travel_square',
   'background_image': 'amtrak.png',
   'text': '',
   'specialProcessing': {'type': 'railTravel', 'destinationSquare': 13}},
  {'

In [10]:
len(hitech_layout['layout'])

42

In [11]:
hitech_layout['layout'][4] #['specialProcessing']

{'number': 5,
 'name': 'FMC',
 'type': 'occupation_entrance_square',
 'background_image': 'occupation_entrance_square.png',
 'image': 'fmc-logo small.jpg',
 'text': '',
 'specialProcessing': {'type': 'enterOccupation',
  'destinationOccupation': 'FMC'}}